## 一些工具函数

In [1]:
def strQ2B(ustring):
    """把字符串全角转半角"""
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return ''.join(ss)

def clean_str(input):

    input = input.replace(",", "，")
    input = input.replace("\xa0", "")
    input = input.replace("\b", "")
    input = input.replace('"', "")
    input = re.sub("\t|\n|\x0b|\x1c|\x1d|\x1e", "", input)
    input = input.strip()
    input = re.sub('\?\?+','',input)
    input = re.sub('\{IMG:.?.?.?\}','',input)
    input = re.sub('\t|\n','', input)
    # pattern1 = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')  # 剔除链接
    pattern1 = re.compile(
        r'((http|ftp|https)://)?(([a-zA-Z0-9\._-]+\.[a-zA-Z]{2,6})|([0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}))(:[0-9]{1,4})*(/[a-zA-Z0-9\&%_\./-~-]*)?')

    pattern2 = re.compile("\{IMG.*?\}")  # 剔除{IMG:1}{IMG:2}等等
    # pattern3 = re.compile("（.*?\）") # 剔除括号等等
    # pattern4 = re.compile("《.*?\》")  # 剔除括号等等
    # pattern5 = re.compile("【.*?】")  # 删除括号内容
    pattern6 = re.compile("\?{2,}")  # 删除多个问号
    pattern7 = re.compile(
        "[\w!#$%&'*+/=?^_`{|}~-]+(?:\.[\w!#$%&'*+/=?^_`{|}~-]+)*@(?:[\w](?:[\w-]*[\w])?\.)+[\w](?:[\w-]*[\w])?")  # 邮箱
    pattern8 = re.compile("0\d{2}-\d{8}|0\d{3}-\d{7}|\d{5}-\d{5}|\d{3}-\d{3}-\d{4}")  # 剔除电话
    pattern9 = re.compile("(20\d{2}([\.\-/|年月\s]{1,3}\d{1,2}){2}日?(\s?\d{2}:\d{2}(:\d{2})?)?)|(\d{1,2}\s?(分钟|小时|天)前)")  # 日期
    pattern10 = re.compile("<.*?>")
    punct = string.punctuation 
    pattern11 = re.compile("[^\u4e00-\u9fa5^a-z^A-Z^0-9%s]+" % punct)
    pattern12 = re.compile('^[?：！*]', re.S)
    pattern13 = re.compile("#")  # 删除#
    pattern14 = re.compile("\(\)|{}")  # 删除多个kuohao

    # shunxu
    pattern = [pattern11, pattern10, pattern2, pattern1, pattern6, pattern7, pattern8, pattern9, pattern14]
    # pattern = [pattern11, pattern10, pattern2, pattern1, pattern6, pattern7, pattern8, pattern14]
    pattern_2 = [pattern12, pattern13]


    # pattern_2 = [pattern4, pattern5, pattern12]
    def clean_zh(text):
        '''清洗文本，保证语句通顺(关于小数点的问题无法处理)'''
        # text = text.replace("（", "(").replace("）", ")")
        # punct = string.punctuation + punctuation
        # punct = "".join([c for c in punct if c not in [".", "、", "%", "“", "”", "(", ")", "！", "。", "？"]])
        # text = re.sub(r"[%s]+" % punct, " ", text)
        # 将引号替换
        # text = re.sub(r"[%s]+" % "“”()", "", text)
        # text = re.sub(r"[%s]+" % "：", " ", text)
        # 多个空格替换成一个
        # text = re.sub('/{2,}', '', text)
        # text = re.sub('\|{2,}', '', text)
        text = re.sub('window.public=.*\(window[,，]document\);', ' ', text)
        text = re.sub('varcontentConEle=.*AD_SURVEY_Add_AdPos\(\"42974\"\);', ' ', text)
        text = re.sub('&nbsp;|&quot;', '', text)
        text = re.sub('　+', ' ', text)
        text = re.sub(' +', ' ', text)
        text = re.sub('%+', '%', text)
        text = re.sub('#+', '#', text)

        # add ccp3
        text = re.sub("\[.*?\]", '', text)
        text = re.sub(",+", '，', text)

        #add
        text = text.replace("\xa0", "")
        text = text.replace("\b", "")
        text = re.sub("\t|\n|\x0b|\x1c|\x1d|\x1e", "", text)

        pattern = [(r'&amp;', '&'), (r'&lt;', '<'), (r'&gt;', '>'), (r'&quot;', ''),
               (r'&nbsp;', ''), (r'br/', ''), (r'&', '&')]

        # 根据pattern清除一些特殊字符
        for p in pattern:
            text = re.sub(p[0], p[1], text)
        
        html1 = re.compile(r'(https?://)([\da-z\.-]+)\.([a-z\.]{2,6})([/\w \.-]*)*/?')
        html2 = re.compile(r'(www)\.([\da-z\.-]+)\.([a-z\.]{2,6})([/\w \.-]*)*/?')

        text = re.sub(html1, ' ', text)
        text = re.sub(html2, ' ', text)
        return text
    input=clean_zh(input)
    return input
    
def juhao(x):
    if x=="":
        return x
    elif x[-1] not in '.。？！!?':
        return x+'。'
    else:
        return x

def loadStopWords(path):
    stopwords = []
    # 加载停止词
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            stopwords.append(line.strip())
    return stopwords



def ngram(news,stopwords):
    if news['title_cut']!='':
        fenci = news['title_cut'].split(',')#########词列表
    else:
        fenci = [word.strip() for word in jieba.cut(news['text'][:int(len(news['text'])/4)]) if (word.strip() not in stopwords and word.strip())]

    text = news['title'] + '\n' + news['text']############title和content之间用\n分割开，，记得 text
    result = copy.deepcopy(fenci)
    length = len(fenci)
    appear = []
    for i in range(length):
        if i < length - 1:
            combine = fenci[i] + fenci[i + 1]#######相邻两个词组成一个词 2-gram，用空格分一下
            count = text.count(combine)
            if count > 1 and combine not in appear:
                appear.append(combine)
                result.append(combine)

            combine = fenci[i] + ' ' + fenci[i + 1]#######相邻两个词组成一个词 2-gram，用空格分一下
            count = text.count(combine)
            if count > 1 and combine not in appear:############如果这个词在title+content中出现的次数大于等于2次，那么就认为这是个新词
                appear.append(combine)
                result.append(combine)

            if i < length - 2:
                ## 分词合并
                combine = fenci[i] + fenci[i + 1] + fenci[i + 2]
                count = text.count(combine)
                if count > 1 and combine not in appear:
                    appear.append(combine)
                    result.append(combine)

                ## 分词加空格合并
                combine = fenci[i] + ' ' + fenci[i + 1] + fenci[i + 2]#######相邻三个词组成一个词 3-gram，用空格分一下
                count = text.count(combine)
                if count > 1 and combine not in appear:
                    appear.append(combine)
                    result.append(combine)

                ## 分词加空格合并
                combine = fenci[i] + fenci[i + 1] + ' ' + fenci[i + 2]
                count = text.count(combine)
                if count > 1 and combine not in appear:
                    appear.append(combine)
                    result.append(combine)
    return ';'.join(result)#############result是分好了词 且进行了ngram的结果

def get_cixing(s):
    tokens = s.split(';')
    cixing = []
    for word in tokens:###############词性标注！！！！！！！[a,b,c]   [a的词性,b的词性,c的词性]
        try:
            p = next(pseg.cut(word)).flag
            cixing.append(p)
        except:
            print(word, s)
    if len(tokens) != len(cixing):
        print(cixing,'*****************')
        print(tokens,"*****************")
        assert len(tokens) == len(cixing), '长度不匹配'
    return ';'.join(cixing)

#     print(cixing,'*****************')
#     print(tokens,"*****************")
#     assert len(tokens) == len(cixing), '长度不匹配'
#     return ';'.join(cixing)

def add_bert_to_tokens(all_df):
    result = []
    for i in tqdm(range(all_df.shape[0])):
        news = all_df.iloc[i,:]
        all_tokens = news['tokens_with_sw']############原始文本分词（title+content）
        if not pd.isnull(news['tokens']):
            tokens = news['tokens'].split(';')#######所有实体（bert的、官方的正确实体、n-gram分词的）取set
            for word in tokens:######（bert的、官方的正确实体、n-gram分词的）
                if word not in all_tokens:###########非原始的词
                    all_tokens = '/split' + word + all_tokens + '/split' + word #/split A BCD /split A  
                    ##### '/split' E  /split A BCD /split A   '/split' E
        result.append(all_tokens)

    return result

# lda和kmeans

In [2]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans

model_path='temp2'
data_gen='temp2'
# 训练LDA主题分类模型
def train_lda(all_df,  n_topics=15):
    ## 使用不包含停止词的分词结果
    corpus = all_df['tokens']
    cnt = CountVectorizer()
    cntIf = cnt.fit_transform(corpus)

    lda_path = os.path.join(model_path, 'lda.pkl')

    ## 使用LDA主题模型进行分类
    lda = LatentDirichletAllocation(n_components=n_topics, max_iter=150)
    print("正在训练LDA主题模型...")
    lda_pred = lda.fit_transform(cntIf)
    lda_classes = np.argmax(lda_pred, axis=1)
    ## 保存模型
    with open(lda_path, 'wb') as f:
        pickle.dump(lda, f)
    print("LDA主题模型已保存...")

    return lda_classes

# 训练KMeans分类器
def train_kmeans(all_df,  n_clusters=15):
    ## 准备训练集
    cluster_train = []
    newsid = all_df['doc_id']
    doc2vec = Doc2Vec.load(os.path.join(model_path, 'doc2vec.model'))

    for ID in newsid:
        cluster_train.append(doc2vec[ID])

    kmeans_path = os.path.join(model_path, 'kmeans.pkl')


    cluster = KMeans(n_clusters=n_clusters)
    print("正在训练KMeans聚类结果...")
    kmeans_classes = cluster.fit_predict(cluster_train)
    ## 保存模型
    with open(kmeans_path, 'wb') as f:
        pickle.dump(cluster, f)
    print("KMean聚类模型已保存...")

    return kmeans_classes


# 训练tsvd主题
def train_tsvd(all_df,  n_topics=15):
    ## 使用不包含停止词的分词结果
    corpus = all_df['tokens']
    tfidf_path = os.path.join(model_path, 'Tfidf.pkl')
    f = open(tfidf_path, 'rb')
    tfidf_model = pickle.load(f)
    f.close()
    print("开始构建tsvd矩阵")
    tfidf_all=tfidf_model.transform(corpus)
    tsvd_pred=decomposition.TruncatedSVD(n_components=15).fit_transform(tfidf_all)
    tsvd_col=['lsi_'+str(i) for i in range(len(tsvd_pred[0]))]
    tsvd_pred=pd.DataFrame(tsvd_pred,columns=tsvd_col)
    
    return tsvd_pred


# 训练w2v和tfidf还有doc2vec

In [3]:
# -*- coding: utf-8 --*--
# @Author: Zessay
# @time: 2019.05.05 20:21
# @File: train_models.py
# @Software: PyCharm

import pandas as pd
import numpy as np
import pickle
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans


'''
# 读取生成的分词文件给后面模型训练使用
token_path = '../data/gen/all_tokens.csv'
all_df = pd.read_csv(token_path)
'''


# 定义训练TFIDF的模型
def train_tfidf(all_df):
    tfidf_tokens = all_df['tokens'].values
    tfidf_model = TfidfVectorizer()
    print("正在训练TFIDF模型...")
    tfidf_model = tfidf_model.fit(tfidf_tokens)

    with open(os.path.join(model_path, 'Tfidf.pkl'), 'wb') as f:
        pickle.dump(tfidf_model, f)
    print("TFIDF模型已保存...")

# 定义训练word2vec的模型
def train_word2vec(all_df):
    sentences = [line.strip().split('/split') for line in all_df['tokens_with_sw']]
    # 训练200维的词向量
    word2vec = Word2Vec(size=200, window=5, min_count=1, iter=20,workers=8)
    word2vec.build_vocab(sentences)
    print("正在训练Word2Vec模型...")
    word2vec.train(sentences, total_examples=word2vec.corpus_count, epochs=word2vec.epochs)
    word2vec.save(os.path.join(model_path, 'word2vec.model'))
    print("Word2Vec模型已保存...")

# 定义训练doc2vec的模型
def train_doc2vec(all_df):
    newsid = all_df['doc_id']
    sentences = [line.strip().split('/split') for line in all_df['tokens_with_sw']]
    # 构建文章向量的语料
    documents = [TaggedDocument(doc, [ID]) for doc, ID in zip(sentences, newsid)]
    doc2vec = Doc2Vec(vector_size=200, window=5, min_count=1, epochs=20)
    doc2vec.build_vocab(documents)
    print("正在训练Doc2Vec模型...")
    doc2vec.train(documents, total_examples=doc2vec.corpus_count, epochs=doc2vec.epochs)
    doc2vec.save(os.path.join(model_path, 'doc2vec.model'))
    print("Doc2Vec模型已保存...")


def train_all_models():
    all_df = pd.read_csv(os.path.join(data_gen, 'all_tokens.csv'))
    # 训练TFIDF
    train_tfidf(all_df)
    # 训练word2vec
    train_word2vec(all_df)
    # 训练doc2vec
    train_doc2vec(all_df)


# 数据预处理

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import os
import math
import re
import datetime
import sys, getopt


import jieba
import re
import pandas as pd
import re
import os
import sys
import numpy as np
import string
from random import choice
import jieba



    
train_df_r2 = pd.read_csv("data/Round2_train_huibiao.csv", encoding="utf-8-sig")
test_df_r2 = pd.read_csv("data/Round2_Test.csv", encoding="utf-8-sig")

train_df_r1 = pd.read_csv("data/r1_train_hand.csv", encoding="utf-8-sig")
test_df_r1 = pd.read_csv("data/Round1_Test_Data.csv", encoding="utf-8-sig")

train_df=pd.concat([train_df_r2,train_df_r1],axis=0)
test_df=pd.concat([test_df_r2,test_df_r1],axis=0)

train_df['title']=train_df['title'].fillna('')
test_df['title']=test_df['title'].fillna('')

train_df['text'] =  train_df['text'].fillna('')
test_df['text'] =   test_df['text'].fillna('')

train_df[['title']]=train_df[['title']].applymap(lambda x:juhao(x))
test_df[['title']]=test_df[['title']].applymap(lambda x:juhao(x))


train_df['text'] = train_df['text'].apply(clean_str)
test_df['text'] = test_df['text'].apply(clean_str)

train_df['title'] = train_df['title'].apply(clean_str)
test_df['title'] = test_df['title'].apply(clean_str)
# 所有的非中文英文数字符号
additional_chars = set()
for t in list(train_df.text) + list(test_df.text) +list(train_df.title) + list(test_df.title):
    additional_chars.update(re.findall(u'[^\u4e00-\u9fa5a-zA-Z0-9\*]', t))

# 一些需要保留的符号
extra_chars = set("!#$%&\()*+,-./:;<=>?@[\\]^_`{|}~！#￥%&？《》{}“”，：‘’。（）·、；【】")
additional_chars = additional_chars.difference(extra_chars)
def remove_additional_chars(input):
    for x in additional_chars:
        input = input.replace(x, "")
    return input
train_df["text"] = train_df["text"].apply(remove_additional_chars)
test_df["text"] = test_df["text"].apply(remove_additional_chars)
train_df["title"] = train_df["title"].apply(remove_additional_chars)
test_df["title"] = test_df["title"].apply(remove_additional_chars)

rawwords = loadStopWords('stopwords.txt')
stopwords = rawwords################情感词和普通词的停用词词典？


train_len=len(train_df)
test_len=len(test_df)
train_r2_len=len(train_df_r2)
test_r2_len=len(test_df_r2)

# 生成去停用词和不去停用词的数据

In [ ]:
import multiprocessing
import jieba
import jieba.posseg as pseg
import pandas as pd
import gc
import os
import re
import codecs
import copy
import numpy as np
from tqdm import tqdm
## 将两组数据进行合并
print("合并训练集和测试集...")
all_df = pd.concat([train_df[['id', 'title', 'text']] ,test_df], axis=0, ignore_index=True)
all_df.columns=['id','title','content']
all_df['text']=all_df['title']+all_df['content']
############训练集和测试集合并了

jieba.load_userdict("all_entities_bert_train.txt")
all_ent_bert_train=pd.read_csv("all_entities_bert_train.txt",header=None)
all_ent_bert_train=list(set([i for i in all_ent_bert_train[0]]))

try:
    all_ent_bert_train.remove(np.nan)
except:
    pass
## 加载bert的结果
# bert_train = pd.read_csv("data/train_fu.csv")
# bert_test=pd.read_csv("data/submit.csv")
# bert_all=pd.concat([bert_train['unknownEntities'],bert_test['unknownEntities']],axis=0).reset_index(drop=True)
all_df['bert_result']=all_df['text'].apply(lambda x:";".join([i for i in all_ent_bert_train if i in x]))
# all_df['title_cut'] = all_df['title'].apply(lambda s: ','.join([word.strip() for word in jieba.cut(s) if (word.strip() not in stopwords and word.strip())]))
print("cut end")
# title = []
# for i in tqdm(range(all_df.shape[0])):
#     news = all_df.iloc[i, :]
#     result = ngram(news,stopwords)#################进行了n-gram（2、3-gram），词和词之间以逗号分割，，，是title的分词结果，，如果title的分词结果在text中出现了两次以上 那么就认为这是一个新词
#     title.append(result)
# all_df['title_ngram'] = title############这里虽然写了title_ngram但实际上是 title+content

all_unknownEntities=[i for i in train_df['unknownEntities']]+[np.nan for i in range(len(test_df))]
all_df['unknownEntities']=all_unknownEntities


train_df = all_df[:len(train_df)]################训练集和测试集分割出来
test_df = all_df[len(train_df):]####各找各妈
train_df['unknownEntities']=train_df['unknownEntities'].fillna('')
train_df['bert_result']=train_df['bert_result'].fillna('')
# train_df['title_ngram']=train_df['title_ngram'].fillna('')
test_df['bert_result']=test_df['bert_result'].fillna('')
# test_df['title_ngram']=test_df['title_ngram'].fillna('')

# train_df['bert_and_title'] = train_df['bert_result'] + ';' + train_df['unknownEntities'] + ';' + train_df['title_ngram']
train_df['bert_and_title'] = train_df['bert_result'] + ';' + train_df['unknownEntities']
#########gt_entity猜想应该是官方给的正确实体，，，，bert_result的预测结果，还有title+content的分词结果
test_df['bert_and_title'] = test_df['bert_result'] 

all_df = pd.concat([train_df, test_df], axis=0, sort=False, ignore_index=True)


all_df['tokens'] = all_df['bert_and_title'].apply(
    lambda s: ';'.join([w.strip() for w in set(s.split(';')) if w!='' and len(w.strip()) > 1]))#######所有实体（bert的、官方的正确实体、n-gram分词的）取set
    ######   tokens理解成所有实体/所有单词 把，，，然后按照  A,B,C这样弄成一个字符串
all_df['tokens']=all_df['tokens'].apply(lambda x:'无' if x=='' else x)

all_df['cixing'] = all_df['tokens'].apply(lambda s: get_cixing(s) if not pd.isnull(s) else s)################A,B,C  A的词性,B的词性,C的词性

# all_df.drop(['bert_result', 'title_cut', 'title_ngram', 'bert_and_title'], axis=1, inplace=True)
all_df.drop(['bert_result', 'bert_and_title'], axis=1, inplace=True)
################id，text，title，content，title_ngram,tokens,cixing

all_df['tokens_with_sw'] = all_df['text'].apply(lambda s: '/split'.join(jieba.cut(s)))############原始文本分词
all_df['tokens_with_sw'] = add_bert_to_tokens(all_df)###############这个 是   有停止词的 tokens_with_stopwords


all_df['doc_id']=[i for i in range(train_len+test_len)]


# 训练TFIDF
train_tfidf(all_df)
pool = multiprocessing.Pool(processes=15)
print("start model make")


# 训练word2vec
pool.apply_async(train_word2vec, [all_df])
# 训练doc2vec
pool.apply_async(train_doc2vec, [all_df])

pool.close()
pool.join()

pool = multiprocessing.Pool(processes=15)
####lda kmeans tsvd
model_fea=[pool.apply_async(train_lda, [all_df]),pool.apply_async(train_kmeans, [all_df]),pool.apply_async(train_tsvd, [all_df])]
pool.close()
pool.join()



all_df['lda_classes'] = model_fea[0].get()
all_df['kmeans_classes'] = model_fea[1].get()
lsi_df=model_fea[2].get()
all_df=pd.concat([all_df,lsi_df],axis=1)

all_df.to_csv("temp2/all_tokens.csv",index=None)






In [ ]:

# bert_dict=pd.read_csv("data/submit.csv",usecols=['unknownEntities'])
# fu_dict=pd.read_csv("data/train_fu.csv",usecols=['unknownEntities'])
# bert_dict=pd.DataFrame(list(set([i for i in bert_dict['unknownEntities']]+[i for i in fu_dict['unknownEntities']]+[i for i in train_df['unknownEntities']])))
# bert_dict.to_csv("temp2/bert_dict.txt",index=None,header=None)
# jieba.load_userdict("temp2/bert_dict.txt")

# 特征生成

## 下面是图特征

### 根据在tokens中的共现关系来构建边

In [ ]:
import networkx as nx
from networkx.readwrite.gpickle import write_gpickle, read_gpickle
import itertools
graph = nx.Graph()
for row_index,row_ent in enumerate(all_df['tokens']):
    if len(row_ent.split(';'))==1:
        graph.add_node(row_ent.split(';')[0]) 
        continue
    row_ent_com=itertools.combinations(row_ent.split(';'),2)
    for single_com in row_ent_com:
        ziped = [(single_com[0], single_com[1])]
        graph.add_edges_from(ziped)
    if row_index%1000==0:
        print(row_index)
print('finish generate test graph, temporaray graph size ->', graph.number_of_nodes(), graph.number_of_edges())

### 图特征函数

In [ ]:
%%time

# # ShortestPath ####两个节点之间的最短路径
def calculate_shortestpath(row):
    graph.remove_edge(row[0], row[1])
    if nx.has_path(graph, row[0], row[1]):
        shortest_path = len(nx.shortest_path(graph, row[0], row[1]))
    else:
        shortest_path = -1
    graph.add_edge(row[0], row[1])
    return [shortest_path]
def gen_degrees(graph):
    max_degrees = {}
    edges = graph.edges()
    for edge in edges:
        for n in edge:
            max_degrees[n] = max_degrees.get(n, 0) + 1
    return max_degrees

def gen_components(graph):
    max_components = {}
    components = nx.connected_components(graph)
    for component in components:
        for n in component:
            max_components[n] = max(max_components.get(n, 0), len(component))
    return max_components

def gen_hits(graph):
    hits_h, hits_a = nx.hits(graph, max_iter=500)
    return hits_h, hits_a

################################single_edge
# Neighbour
def calculate_single_neighbour(tokens):
    tokens = tokens.split(';')
    neighbor_list=[]
    for word in tokens:###############词性标注！！！！！！！[a,b,c]   [a的词性,b的词性,c的词性]

        neighbor_1 = set(graph.neighbors(word))
        neighbor_num=len(neighbor_1)
        neighbor_list.append(str(neighbor_num))

    if len(tokens) != len(neighbor_list):
        assert len(tokens) == len(neighbor_list), '长度不匹配'
    return ';'.join(neighbor_list)
        

def calculate_single_hits_h(tokens):
    tokens = tokens.split(';')
    hits_h_1_list=[]
    for word in tokens:###############词性标注！！！！！！！[a,b,c]   [a的词性,b的词性,c的词性]

        hits_h_1 = hits_h[word] * 1e6
        hits_h_1_list.append(str(hits_h_1))

    if len(tokens) != len(hits_h_1_list):
        assert len(tokens) == len(hits_h_1_list), '长度不匹配'
    return ';'.join(hits_h_1_list)
        

def calculate_single_hits_a(tokens):
    tokens = tokens.split(';')
    hits_a_1_list=[]
    for word in tokens:###############词性标注！！！！！！！[a,b,c]   [a的词性,b的词性,c的词性]

        hits_a_1 = hits_a[word] * 1e6
        hits_a_1_list.append(str(hits_a_1))

    if len(tokens) != len(hits_a_1_list):
        assert len(tokens) == len(hits_a_1_list), '长度不匹配'
    return ';'.join(hits_a_1_list)
        

def calculate_single_statistics_degrees(tokens):
    tokens = tokens.split(';')
    max_degrees_list=[]
    for word in tokens:###############词性标注！！！！！！！[a,b,c]   [a的词性,b的词性,c的词性]
        try:
            max_degrees_list.append(str(max_degrees[word]))
        except:
            max_degrees_list.append("0")
    if len(tokens) != len(max_degrees_list):
        assert len(tokens) == len(max_degrees_list), '长度不匹配'
    return ';'.join(max_degrees_list)
        


def calculate_single_statistics_components(tokens):
    tokens = tokens.split(';')
    components_list=[]
    for word in tokens:###############词性标注！！！！！！！[a,b,c]   [a的词性,b的词性,c的词性]

        components_list.append(str(max_components[word]))

    if len(tokens) != len(components_list):
        assert len(tokens) == len(components_list), '长度不匹配'
    return ';'.join(components_list)
        
def calculate_single_pagerank(tokens):
    tokens = tokens.split(';')
    pagerank_list=[]
    for word in tokens:###############词性标注！！！！！！！[a,b,c]   [a的词性,b的词性,c的词性]

        pagerank_list.append(str(page_rank[word] * 1e6 ))

    if len(tokens) != len(pagerank_list):
        assert len(tokens) == len(pagerank_list), '长度不匹配'
    return ';'.join(pagerank_list)
        
pool = multiprocessing.Pool(processes=4)
print("start model make")


####lda kmeans tsvd
model_graph=[pool.apply_async(gen_degrees, [graph]),pool.apply_async(nx.pagerank_scipy, [graph]),pool.apply_async(gen_components, [graph]),pool.apply_async(gen_hits, [graph])]

pool.close()
pool.join()
    
    
max_degrees = model_graph[0].get()
page_rank = model_graph[1].get()
max_components = model_graph[2].get()
hits_h, hits_a = model_graph[3].get()

def all_grapha_process(part_index):
    one_part_df=pd.DataFrame()
    one_part_df['neighbour'] = all_df_r2['tokens'].iloc[part_index].apply(lambda s: calculate_single_neighbour(s) if not pd.isnull(s) else s)################A,B,C  A的词性,B的词性,C的词性
    one_part_df['hits_h'] = all_df_r2['tokens'].iloc[part_index].apply(lambda s: calculate_single_hits_h(s) if not pd.isnull(s) else s)################A,B,C  A的词性,B的词性,C的词性
    one_part_df['hits_a'] = all_df_r2['tokens'].iloc[part_index].apply(lambda s: calculate_single_hits_a(s) if not pd.isnull(s) else s)################A,B,C  A的词性,B的词性,C的词性
    one_part_df['degrees'] = all_df_r2['tokens'].iloc[part_index].apply(lambda s: calculate_single_statistics_degrees(s) if not pd.isnull(s) else s)################A,B,C  A的词性,B的词性,C的词性
    one_part_df['components'] = all_df_r2['tokens'].iloc[part_index].apply(lambda s: calculate_single_statistics_components(s) if not pd.isnull(s) else s)################A,B,C  A的词性,B的词性,C的词性
    one_part_df['pagerank'] = all_df_r2['tokens'].iloc[part_index].apply(lambda s: calculate_single_pagerank(s) if not pd.isnull(s) else s)################A,B,C  A的词性,B的词性,C的词性
    return one_part_df
    
all_df_train=all_df[:train_len]
all_df_test=all_df[train_len:]
all_df_r2=pd.concat([all_df_train[:train_r2_len],all_df_test[:test_r2_len]],axis=0).reset_index(drop=True)

pool = multiprocessing.Pool(processes=40)
grapha_fea=[]

split_size=int(len(all_df_r2)/40)
all_index=list(range(len(all_df_r2)))
df_split_index=[all_index[i:i+split_size] for i in range(0,len(all_index),split_size)]  

for part_id,part_index in enumerate(df_split_index):    
    grapha_fea.append(pool.apply_async(all_grapha_process, [part_index]))
    print(part_id)
pool.close()
pool.join()
grapha_fea=[i.get() for i in grapha_fea]
grapha_df=pd.concat(grapha_fea,axis=0).reset_index(drop=True)
all_df_r2=pd.concat([all_df_r2,grapha_df],axis=1)
print("模型训练完成")

train_df = all_df_r2[:train_r2_len]################训练集和测试集分割出来
test_df = all_df_r2[train_r2_len:]####各找各妈
train_df.to_csv("temp2/train_cut.csv",index=None)
test_df.to_csv("temp2/test_cut.csv",index=None)b

In [ ]:
train_df.columns

# 下面是其他特征

In [ ]:
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import pickle
import jieba
import jieba.posseg as pseg
import jieba.analyse
import numpy as np
import pandas as pd
import json
import os
import re
import codecs
import math
from gensim.models import Word2Vec, Doc2Vec, KeyedVectors
from tqdm import tqdm

import gc
import math

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')



def load_models( df_):
    # 获取模型的位置
    tfidf_path = os.path.join(model_path, 'Tfidf.pkl')
    w2v_path = os.path.join(model_path, 'word2vec.model')
    d2v_path = os.path.join(model_path, 'doc2vec.model')
    # 获取TFIDF模型
    try:
        f = open(tfidf_path, 'rb')
    except:
        train_tfidf(df_)
        f = open(tfidf_path)
    finally:
        tfidf_model = pickle.load(f)
        f.close()


    # 加载word2vec和doc2vec模型
    try:
        w2v_model = Word2Vec.load(w2v_path)
    except:
        train_word2vec(df_)
        w2v_model = Word2Vec.load(w2v_path)

    try:
        d2v_model = Doc2Vec.load(d2v_path)
    except:
        train_doc2vec(df_)
        d2v_model = Doc2Vec.load(d2v_path)

    return tfidf_model, w2v_model, d2v_model

# 计算单词与词向量的余弦相似度和欧氏距离
## 计算余弦相似度
def Cosine(wordvec, docvec):
    wordvec, docvec = np.array(wordvec), np.array(docvec)
    return wordvec.dot(docvec) / (math.sqrt((wordvec**2).sum()) * math.sqrt((docvec**2).sum()))

## 计算欧式距离
def Euclidean(wordvec, docvec):
    wordvec, docvec = np.array(wordvec), np.array(docvec)
    return math.sqrt(((wordvec-docvec)**2).sum())

# 判断是否是数字
def is_number(x):
    try:
        float(x)
        return True
    except:
        return False


def extract_word_property(news, tfidf_model, w2v_model, d2v_model, train=True):
    tmp = pd.DataFrame()
    text = news['title'] + '\n' + news['content']
    # words = pseg.cut(text)
    # print(list(zip(*words)))
    # li = list(zip(*words))
    # print(li[0])
    assert len(news['tokens'].split(';')) == len(news['cixing'].split(';')), '长度不匹配'

    tmp['word'] = news['tokens'].split(';')
    tmp['word'] = tmp['word'].apply(lambda w: w.strip())
    
    tmp['cixing'] = news['cixing'].split(';')

    tmp['neighbour'] = news['neighbour'].split(';')
    tmp['hits_h'] = news['hits_h'].split(';')
    tmp['hits_a'] = news['hits_a'].split(';')
    tmp['degrees'] = news['degrees'].split(';')
    tmp['components'] = news['components'].split(';')
    tmp['pagerank'] = news['pagerank'].split(';')

    tmp[['neighbour','hits_h','hits_a','degrees','components','pagerank']]=tmp[['neighbour','hits_h','hits_a','degrees','components','pagerank']].astype(np.float64)

    tmp['doc_id'] = news['doc_id']
    tmp['lda_classes'] = news['lda_classes']
    tmp['kmeans_classes'] = news['kmeans_classes']
    tsvd_col=[i for i in list(news.index) if 'lsi' in i]
    tsvd=[i for i in news[tsvd_col].values] 
    tsvd=[tsvd for tsvd_row_num in range(len(tmp))]
    tsvd=pd.DataFrame(tsvd,columns=tsvd_col)
    
    tmp=pd.concat([tmp,tsvd],axis=1)
    # 添加词频的列
    tmp.dropna(axis=0, subset=['word'], inplace=True)
    tmp['tf'] = tmp['word'].apply(lambda w: text.count(w))
    tmp.drop_duplicates(inplace=True)
    

    stopwords = loadStopWords('stopwords.txt')
    no_core_words = loadStopWords('no_core_stopword.txt')
    no_core_words.sort(key=lambda x:len(x),reverse=True)##########停用词从大到小排序
    def no_core_word_in_word(word):
        for no_core in no_core_words:
            if no_core in word:
                return no_core
        return '没有停用'
        
    def cut_sent(para):
        para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
        para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
        para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
        para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
        # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
        para = para.rstrip()  # 段尾如果有多余的\n就去掉它
        # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
        return para.split("\n")

    # 添加是否为停用词的标记，或者为空，或者为纯数字
    tmp['flag'] = tmp['word'].apply(lambda w: int((w in stopwords) or (is_number(w))))
    # 删除停用词，取出非停用词
    tmp = tmp[tmp['flag'] == 0]
    tmp.drop(['flag'], axis=1, inplace=True)

    tmp['stop_in_word'] = tmp['word'].apply(lambda w: no_core_word_in_word(w))###判断一个词里是不是有停用词

    sen_list=cut_sent(news['content'])
    sen_num=len(sen_list)
    def is_in_first_sent(word):
        if word in sen_list[0]:
            return 1
        else:
            return 0
    def is_in_last_sent(word):
        if word in sen_list[-1]:
            return 1
        else:
            return 0
    def is_in_other_sent(word):
        if word not in sen_list[0] and word not in sen_list[-1]:
            return 1
        else:
            return 0
    tmp['is_in_first_sent'] = tmp['word'].apply(lambda w: is_in_first_sent(w))###判断一个词是不是在第一句
    tmp['is_in_last_sent'] = tmp['word'].apply(lambda w: is_in_last_sent(w))###判断一个词里是不是在最后一句
    tmp['is_in_other_sent'] = tmp['word'].apply(lambda w: is_in_other_sent(w))###判断一个词里是不是在其他句子（或标题）
    tmp['sen_num']=sen_num
    
    ###################组内特征
    ######组内相词频特征

    
    tmp['word_count_xiangdui_max']=tmp['word'].apply(lambda w: text.count(w))####相对于最大的

    tmp['word_count_xiangdui_max']=tmp['word_count_xiangdui_max']/max(tmp['word_count_xiangdui_max'].values)
    tmp['word_count_xiangdui_min']=tmp['word'].apply(lambda w: text.count(w))
    tmp['word_count_xiangdui_min']=tmp['word_count_xiangdui_min']/min(tmp['word_count_xiangdui_min'].values)
    tmp['word_count_group_max']=tmp['word'].apply(lambda w: text.count(w))
    tmp['word_count_group_max']=max(tmp['word_count_group_max'].values)
    tmp['word_count_group_min']=tmp['word'].apply(lambda w: text.count(w))
    tmp['word_count_group_min']=min(tmp['word_count_xiangdui_min'].values)
    tmp['word_count_group_avg']=tmp['word'].apply(lambda w: text.count(w))
    tmp['word_count_group_avg']=sum(tmp['word_count_group_avg'].values)/len(tmp['word_count_group_avg'].values)
    tmp['word_count_group_min_xiangdui_max']=tmp['word'].apply(lambda w: text.count(w))
    tmp['word_count_group_min_xiangdui_max']=min(tmp['word_count_group_min_xiangdui_max'].values)/max(tmp['word_count_group_min_xiangdui_max'].values)
    
    #####组内所有候选词特征
    tmp['all_word_count_group']=tmp['word'].apply(lambda w: len(list(tmp['word'])))####组内有多少候选词


    ######组内公司特征
    def gongsi_count_group(group_word_list):
        gongsi_num=0
        for word in group_word_list:
            if [i for i in ['公司','集团','有限','股份','科技'] if i in word]!=[]:
                gongsi_num+=1
        return gongsi_num

    tmp['gongsi_count_group']=tmp['word'].apply(lambda w: gongsi_count_group(list(tmp['word'])))####组内有多少含有"公司"候选词


    def gongsi_self_div_max_index_group(self_word,group_word_list):
        if [i for i in ['公司','集团','有限','股份','科技'] if i in self_word]==[]:
            return -1
        self_gongsi_first_index=text.find(self_word)
        other_gongsi_last_index=self_gongsi_first_index
        for word in group_word_list:
            if word!=self_word:
                if [i for i in ['公司','集团','有限','股份','科技'] if i in word]!=[]:
                    other_gongsi_last_index=max(text.rfind(word),other_gongsi_last_index)

        return self_gongsi_first_index/(other_gongsi_last_index+2)

    tmp['gongsi_self_div_max_index_group']=tmp['word'].apply(lambda w: gongsi_self_div_max_index_group(w,list(tmp['word'])))####组内有多少含有"公司"候选词

    def gongsi_self_div_min_index_group(self_word,group_word_list):
        if [i for i in ['公司','集团','有限','股份','科技'] if i in self_word]==[]:
            return -1
        self_gongsi_first_index=text.find(self_word)
        other_gongsi_last_index=self_gongsi_first_index
        for word in group_word_list:
            if word!=self_word:
                if [i for i in ['公司','集团','有限','股份','科技'] if i in word]!=[] :
                    other_gongsi_last_index=min(text.find(word),other_gongsi_last_index)

        return self_gongsi_first_index/(other_gongsi_last_index+2)
    tmp['gongsi_self_div_min_index_group']=tmp['word'].apply(lambda w: gongsi_self_div_min_index_group(w,list(tmp['word'])))####组内有多少含有"公司"候选词

    def is_gongsi_in_title(self_word,group_word_list):
        if [i for i in ['公司','集团','有限','股份','科技'] if i in self_word]==[]:
            return -1
        for word in group_word_list:
            if [i for i in ['公司','集团','有限','股份','科技'] if i in word]!=[] :
                if word != self_word and word in news['title']:
                    return 1
        return 0
    tmp['is_gongsi_in_title']=tmp['word'].apply(lambda w: is_gongsi_in_title(w,list(tmp['word'])))####组内有多少含有"公司"候选词

    def is_gongsi_in_first_two_sen(self_word,group_word_list):###self_word
        if [i for i in ['公司','集团','有限','股份','科技'] if i in self_word]==[]:
            return -1
        title_tmp=sen_list[0]
        if len(sen_list)>=2:
            title_tmp+=sen_list[1]
        for word in group_word_list:
            if [i for i in ['公司','集团','有限','股份','科技'] if i in word]!=[] :
                if word != self_word and word in title_tmp:
                    return 1
        return 0
    tmp['is_gongsi_in_first_two_sen']=tmp['word'].apply(lambda w: is_gongsi_in_first_two_sen(w,list(tmp['word'])))####组内有多少含有"公司"候选词


    def gongsi_avg_index(group_word_list):###self_word
        all_first_index=[]
        for word in group_word_list:
            if [i for i in ['公司','集团','有限','股份','科技'] if i in word]!=[] :
                all_first_index.append(text.find(word))
        if all_first_index==[]:
            return -1
        return sum(all_first_index)/(len(all_first_index)+2)
    tmp['gongsi_avg_index']=tmp['word'].apply(lambda w: gongsi_avg_index(list(tmp['word'])))####组内有多少含有"公司"候选词


    def gongsi_max_count(group_word_list):###self_word
        all_gongsi_count=[]
        for word in group_word_list:
            if [i for i in ['公司','集团','有限','股份','科技'] if i in word]!=[] :
                all_gongsi_count.append(text.count(word))
        if all_gongsi_count==[]:
            return -1
        return max(all_gongsi_count)
    tmp['gongsi_max_count']=tmp['word'].apply(lambda w: gongsi_max_count(list(tmp['word'])))####组内有多少含有"公司"候选词


    
    ################
    
    
#     if len(tmp['word'])==0:
#         return pd.Series([np.nan])
    # 增加词的长度特征
    tmp['word_len'] = tmp['word'].apply(lambda w: len(w))
    ## 相对于最大词长的比值
    
    max_len = max(tmp['word_len'].values)
    tmp['word_len_ratio'] = tmp['word_len'].apply(lambda x: x / max_len)
    # 去除词长小于1的
    tmp = tmp[tmp.word_len > 1]

    # 计算频率
    tmp['tf_ratio'] = tmp['tf'] / sum(tmp['tf'])

    ## 获取所有的实体
    feature_name = tfidf_model.get_feature_names()
    # 获取TfIdf
    tfidf_fetures = tfidf_model.transform([news['tokens']])
    tfidf_map = {feature_name[k]: v for k, v in zip(tfidf_fetures.indices, tfidf_fetures.data)}
    tmp['tfidf'] = tmp['word'].map(tfidf_map)
    ## 对TFIDF进行归一化
    tmp['tfidf'] = normalize(tmp['tfidf'].fillna(0).values.reshape(1, -1), 'max')[0]#############按最大的进行归一化！？




    # 是否出现在标题中
    tmp['dai_in_word'] = tmp['word'].apply(lambda w: int((1 if [i for i in ['贷'] if  w.endswith(i)] !=[] else 0)))
    tmp['bao_in_word'] = tmp['word'].apply(lambda w: int((1 if [i for i in ['宝'] if  w.endswith(i)] !=[] else 0)))
    tmp['bi_in_word'] = tmp['word'].apply(lambda w: int((1 if [i for i in ['币'] if  w.endswith(i)] !=[] else 0)))
    tmp['gongsi_in_word'] = tmp['word'].apply(lambda w: int((1 if [i for i in ['公司','科技','控股','集团'] if  w.endswith(i)] !=[] else 0)))
    tmp['zhengquan_in_word'] = tmp['word'].apply(lambda w: int((1 if [i for i in ['证券','债券'] if  w.endswith(i)] !=[] else 0)))
    tmp['kuohao_in_word'] = tmp['word'].apply(lambda w: int((1 if [i for i in [')','）'] if  w.endswith(i)] !=[] else 0)))


    # 获取textrank
    allow_cixing=['a', 'ad', 'ag', 'an', 'b', 'c', 'd',
                'df', 'dg', 'e', 'eng', 'f', 'g', 'h',
                'i', 'j', 'k', 'l', 'm', 'mg', 'mq',
                'n', 'ng', 'nr', 'nrfg', 
                'nrt', 'ns', 'nt', 'nz',
                'o', 'p', 'q', 'r', 'rg',
                'rr', 'rz', 's', 't', 'tg',
                'u', 'ud', 'ug', 'uj', 'ul', 'uv', 'uz',
                'v', 'vd', 'vg', 'vi',
                'vn', 'vq', 'w', 'x', 
                'y', 'z', 'zg']
    tr_score = jieba.analyse.textrank(text, topK=1000, withWeight=True, allowPOS=list(allow_cixing))
    #####参与textrank的词的数量
    
    ####在整篇文章中的textrank排名
    tr_map = {k: v for k, v in tr_score}
    tr_rank_map = {k: rank for rank, (k,v) in enumerate(tr_score)}
    tmp['text_rank'] = tmp['word'].map(tr_map)
    tmp['text_rank_geshu']=len(tr_score)
    tmp['text_rank_rank'] = tmp['word'].map(tr_rank_map)
    # 返回前100
    #tmp = tmp if tmp.shape[0] <= 100 else tmp.iloc[:100, :]

    ######同候选词内包含关系特征 ，最长的那个实体 长度，最短包含的那个实体长度，被包含的词频和包含的词频的比值（按最小被包含关系走）
    all_group_word=list(tmp['word'])
    def baohan_max_len(word):
        all_group_word.sort(key=lambda x:len(x),reverse=True)####从大到小
        for other_word in all_group_word:
            if word in other_word and word!=other_word:
                return len(other_word)
        return -1

    def baohan_min_len(word):
        all_group_word.sort(key=lambda x:len(x))####从小到大排序
        for other_word in all_group_word:
            if word in other_word and word!=other_word:
                return len(other_word)
        return -1

    def baohan_self_div_other_count(word):
        all_group_word.sort(key=lambda x:len(x))####从小到大排序
        for other_word in all_group_word:
            if word in other_word and word!=other_word:
                return text.count(word)/(text.count(other_word)+1)
        return -1

    def baohan_self_div_other_count_reserve(word):
        all_group_word.sort(key=lambda x:len(x),reverse=True)####从大到小排序
        for other_word in all_group_word:
            if word in other_word and word!=other_word:
                return text.count(word)/(text.count(other_word)+1)
        return -1

    tmp['baohan_max_len'] = tmp['word'].apply(lambda w: baohan_max_len(w))
    tmp['baohan_min_len'] = tmp['word'].apply(lambda w: baohan_min_len(w))
    tmp['baohan_self_div_other_count'] = tmp['word'].apply(lambda w: baohan_self_div_other_count(w))###
    tmp['baohan_self_div_other_count_reserve'] = tmp['word'].apply(lambda w: baohan_self_div_other_count_reserve(w))###

    def find_repeat(source,elmt): # The source may be a list or string.
            elmt_index=[]
            s_index = 0;e_index = len(source)
            while(s_index < e_index):
                    try:
                        temp = source.index(elmt,s_index,e_index)
                        elmt_index.append(temp)
                        s_index = temp + 1
                    except ValueError:
                        break
            return elmt_index
        
    def get_avg_pos(wenben,ci):
        all_pos=find_repeat(wenben,ci)
        avg_pos=sum(all_pos)/(len(all_pos)+1)
        return avg_pos


    

    ##文章长度特征
    tmp['text_len'] = len(text)
    tmp['title_len'] = len(news['title'])
    tmp['content_len'] = len(news['content'])
    tmp['title_div_text_len']=len(news['title'])/(len(text)+1)
    #####词在标题中的平均位置 并除以文章长度
    tmp['avg_ocur_title'] = tmp['word'].apply(lambda w: get_avg_pos(news['title'],w)/(len(news['title'])+1))
    #####词在正文中的平均位置 并除以文章长度
    tmp['avg_ocur_content'] = tmp['word'].apply(lambda w: get_avg_pos(news['content'],w)/(len(news['content'])+1))
    

    # 找出第一次出现的位置
    tmp['first_ocur'] = tmp['word'].apply(lambda w: text.find(w) + len(w) - 1)
    # 找出最后一次出现的位置
    reverse_text = text[-1::-1]
    all_len = len(text)
    tmp['last_ocur'] = tmp['word'].apply(lambda w: all_len - (reverse_text.find(w[-1::-1]) + len(w)))
    # 计算词跨度
    tmp['word_distance'] = (tmp['last_ocur'] - tmp['first_ocur']).apply(lambda d: 0 if d < 0 else d)
    # 对词跨度进行归一化
    tmp['word_distance_norm'] = normalize(tmp['word_distance'].values.reshape(1, -1), 'max')[0]
    tmp.drop(['first_ocur', 'last_ocur', 'word_distance'], axis=1, inplace=True)

    # 计算词向量和文档向量的余弦距离
    tmp['Cosine'] = tmp['word'].apply(
        lambda w: np.nan if w not in w2v_model.wv.vocab else Cosine(w2v_model.wv[w], d2v_model[news['doc_id']]))

    # 计算词向量和文档向量的欧式距离
    tmp['Euclidean'] = tmp['word'].apply(
        lambda w: np.nan if w not in w2v_model.wv.vocab else Euclidean(w2v_model.wv[w], d2v_model[news['doc_id']]))

    # 计算皮尔逊相关系数
    tmp['pearson_cor'] = tmp['word'].apply(
        lambda w: np.nan if w not in w2v_model.wv.vocab else stats.pearsonr(w2v_model.wv[w], d2v_model[news['doc_id']])[
            0])
    tmp['pearson_pvalue'] = tmp['word'].apply(
        lambda w: np.nan if w not in w2v_model.wv.vocab else stats.pearsonr(w2v_model.wv[w], d2v_model[news['doc_id']])[
            1])

    # 是否出现在标题中
    tmp['ocur_in_title'] = tmp['word'].apply(lambda w: int((1 if news['title'].find(w) != -1 else 0)))
    # 是否出现在正文中
    tmp['ocur_in_content'] = tmp['word'].apply(lambda w: int((1 if news['content'].find(w) != -1 else 0)))

    # 是否还有数字，是否含有字母
    tmp['has_num'] = tmp['word'].apply(lambda w: int(bool(re.search(r'\d', w))))
    tmp['has_char'] = tmp['word'].apply(lambda w: int(bool(re.search(r'[a-zA-Z]+', w))))

    # 添加共现矩阵信息
    sentences = [news['title']]#########他把标题单独拿成一句话了
    for seq in re.split(r'[\n。？！?!.]', news['content']):
        # 如果开头不是汉字、字母或数字，则去除
        seq = re.sub(r'^[^\u4e00-\u9fa5A-Za-z0-9]+', '', seq)
        # 去除之后，如果句子不为空，则添加进句子中
        if len(seq) > 0:
            sentences.append(seq)

    num_tokens = len(tmp['word'])
    words_list = tmp['word'].tolist()
    arr = np.zeros((num_tokens, num_tokens))
    # 得到共现矩阵
    for i in range(num_tokens):
        for j in range(i + 1, num_tokens):
            count = 0
            for sentence in sentences:
                if (words_list[i] in sentence) and (words_list[j] in sentence):
                    count += 1
            arr[i, j] = count
            arr[j, i] = count

    ## 得到偏度的统计特征
    tmp['coocur_skew'] = stats.skew(arr)
    ## 计算某个单词共现矩阵的方差、峰度
    tmp['coocur_var'] = np.var(arr, axis=0)
    tmp['coocur_mean'] = np.mean(arr, axis=0)
    tmp['coocur_kurt'] = stats.kurtosis(arr, axis=0)
    tmp['coocur_std'] = np.std(arr, axis=0)
    #tmp['coocur_median'] = np.median(arr, axis=0)
    #tmp['coocur_variation'] = stats.variation(arr, axis=0)

    ## 共现矩阵一阶差分
    co_diff1 = np.diff(arr, n=1, axis=1)
    tmp['diff_coocur_mean'] = np.mean(co_diff1, axis=1)
    tmp['diff_coocur_var'] = np.var(co_diff1, axis=1)
    tmp['diff_coocur_std'] = np.std(co_diff1, axis=1)
    #tmp['diff_coocur_median'] = np.median(co_diff1, axis=1)
    tmp['diff_coocur_skew'] = stats.skew(co_diff1, axis=1)
    tmp['diff_coocur_kurt'] = stats.kurtosis(co_diff1, axis=1)

    '''
    ## 共现矩阵二阶差分的统计特性
    co_diff2 = np.diff(arr, n=2, axis=1)
    tmp['diff2_coocur_mean'] = np.mean(co_diff2, axis=1)
    tmp['diff2_coocur_var'] = np.var(co_diff2, axis=1)
    tmp['diff2_coocur_std'] = np.std(co_diff2, axis=1)
    tmp['diff2_coocur_median'] = np.median(co_diff2, axis=1)
    tmp['diff2_coocur_skew'] = stats.skew(co_diff2, axis=1)
    tmp['diff2_coocur_kurt'] = stats.kurtosis(co_diff2, axis=1)
    '''

    # 计算相似度矩阵以及统计信息
    ## sim_tags_arr: 初始化候选词相似度矩阵
    sim_tags_arr = np.zeros((num_tokens, num_tokens))

    for i in range(num_tokens):
        for j in range(i, num_tokens):
            sim_tags_arr[i][j] = 0 if (words_list[i] not in w2v_model.wv.vocab or words_list[
                j] not in w2v_model.wv.vocab) else w2v_model.wv.similarity(words_list[i], words_list[j])
            if i != j:
                sim_tags_arr[j][i] = sim_tags_arr[i][j]
    # 计算单词相似度矩阵的统计信息
    ## 相似度平均值
    tmp['mean_sim_tags'] = np.mean(sim_tags_arr, axis=1)
    ## 相似度矩阵的偏度
    tmp['skew_sim_tags'] = stats.skew(sim_tags_arr, axis=1)
    ## 相似度矩阵的峰值
    tmp['kurt_sim_tags'] = stats.kurtosis(sim_tags_arr, axis=1)
    tmp['var_sim_tags'] = np.var(sim_tags_arr, axis=1)
    tmp['std_sim_tags'] = np.std(sim_tags_arr, axis=1, ddof=1)
    #tmp['median_sim_tags'] = np.median(sim_tags_arr, axis=1)
    #tmp['variation_sim_tags'] = stats.variation(sim_tags_arr, axis=1)

    ## 一阶差分统计特征补充
    diff1 = np.diff(sim_tags_arr, n=1, axis=1)
    tmp['diff_median_sim_tags'] = np.median(diff1, axis=1)
    tmp['diff_var_sim_tags'] = np.var(diff1, axis=1)
    tmp['diff_std_sim_tags'] = np.std(diff1, axis=1)
    ## 相似度矩阵的差分均值
    tmp['diff_mean_sim_tags'] = np.mean(diff1, axis=1)
    ## 相似度矩阵差分的偏度
    tmp['diff_skew_sim_tags'] = stats.skew(diff1, axis=1)
    ## 相似度矩阵差分的峰度
    tmp['diff_kurt_sim_tags'] = stats.kurtosis(diff1, axis=1)

    '''
    ## 二阶差分统计特征
    diff2 = np.diff(sim_tags_arr, n=2, axis=1)
    tmp['diff2_mean_sim_tags'] = np.mean(diff2, axis=1)
    tmp['diff2_median_sim_tags'] = np.median(diff2, axis=1)
    tmp['diff2_var_sim_tags'] = np.var(diff2, axis=1)
    tmp['diff2_std_sim_tags'] = np.std(diff2, axis=1)
    tmp['diff2_skew_sim_tags'] = stats.skew(diff2, axis=1)
    tmp['diff2_kurt_sim_tags'] = stats.kurtosis(diff2, axis=1)
    '''

    # 包含关系矩阵
    in_arr = np.zeros((num_tokens, num_tokens))
    ## 计算包含关系矩阵
    for i in range(num_tokens):
        for j in range(num_tokens):
            if i != j:
                if words_list[i] in words_list[j]:
                    in_arr[i][j] = 1

    ### 被多少个词包含
    tmp['be_include_sum'] = np.sum(in_arr, axis=1)
    tmp['be_include_mean'] = np.mean(in_arr, axis=1)
    tmp['be_include_var'] = np.var(in_arr, axis=1)
    tmp['be_include_std'] = np.std(in_arr, axis=1)
    tmp['be_include_skew'] = stats.skew(in_arr, axis=1)
    tmp['be_include_kurt'] = stats.kurtosis(in_arr, axis=1)

    ### 包含了多少个词
    tmp['include_sum'] = np.sum(in_arr, axis=0)
    tmp['include_mean'] = np.mean(in_arr, axis=0)
    tmp['include_var'] = np.var(in_arr, axis=0)
    tmp['include_std'] = np.std(in_arr, axis=0)
    tmp['include_skew'] = stats.skew(in_arr, axis=0)
    tmp['include_kurt'] = stats.kurtosis(in_arr, axis=0)
    ######应该加一个文章中所出现的所有词的tfidf排名
    
    # 对tfidf排名
    tmp = tmp.sort_values(by='tfidf', ascending=False)
    tmp['tfidf_index'] = range(tmp.shape[0])
    # 对TFIDF索引进行归一化
    tmp['tfidf_index'] = normalize(tmp['tfidf_index'].values.reshape(1, -1), 'max')[0]

    # 对词频排名
    tmp = tmp.sort_values(by='tf', ascending=False)
    tmp['tf_index'] = range(tmp.shape[0])

    # 对词跨度排名
    tmp = tmp.sort_values(by='word_distance_norm', ascending=False)
    tmp['word_distance_norm_index'] = range(tmp.shape[0])
    # tmp['word_distance_norm_index'] = normalize(tmp['word_distance_norm_index'].values.reshape(1, -1), 'max')[0]

    # 对Cosine排名
    tmp = tmp.sort_values(by='Cosine', ascending=False)
    tmp['cosine_index'] = range(tmp.shape[0])

    # 对EUC排名
    tmp = tmp.sort_values(by='Euclidean', ascending=True)
    tmp['euclidean_index'] = range(tmp.shape[0])

    
    # 如果是训练集，获取标签
    if train:
        true_entity = news['unknownEntities'].split(';')
        # print(train_news['doc_id'], true_entity, "文章长度为%d" % all_len)
        tmp['label'] = tmp['word'].apply(lambda w: int(w in true_entity))
        tmp = tmp.sort_values(by=['label', 'tfidf', 'tf'], ascending=False)
    else:
        tmp = tmp.sort_values(['tfidf', 'tf'], ascending=False)

        
        
    return tmp.reset_index(drop=True)


def add_features(train_df, test_df):
    '''
    增加新特征或者对特征进行转换
    :param train_df:
    :param test_df:
    :return:
    '''
    documents=pd.DataFrame()

    documents['text']=all_df['title']+all_df['content']


    def getBayesSmoothParam(origion_rate):
        origion_rate_mean = origion_rate.mean()
        origion_rate_var = origion_rate.var()
        alpha = origion_rate_mean / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
        beta = (1 - origion_rate_mean) / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
        return alpha, beta
    
    all_df_fea = pd.concat([train_df, test_df], axis=0, sort=False, ignore_index=True)
    all_df_fea['idf'] = all_df_fea['word'].map(Counter(all_df_fea['word']))###########被当成候选词的次数~~~~~~
    ## 对词性编码
    lb = LabelEncoder()
    all_df_fea['cixing'] = lb.fit_transform(all_df_fea['cixing'])
    lb2 = LabelEncoder()
    all_df_fea['stop_in_word'] = lb2.fit_transform(all_df_fea['stop_in_word'])
    ####获取单词在所有文章中的频率(先不变换成idf值)
    all_word=set([i for i in all_df_fea['word']])
    id_freq = {}
    i=0
    for doc in documents['text']:
        for word in all_word:
            if word in doc:
                id_freq[word] = id_freq.get(word, 0) + 1
        if i % 1000 == 0:
            print('Documents processed: ', i, ', time: ',
                datetime.datetime.now())
        i += 1
    id_freq = [[word,word_freq] for word,word_freq in id_freq.items()]
    id_freq = pd.DataFrame(id_freq,columns=['word','word_occor_times'])
    id_freq['idf_in_all_text']= id_freq['word_occor_times'].apply(lambda x:math.log(i / x, 2))
    id_freq=id_freq[['word','idf_in_all_text']]

    ######################被当成候选词的概率
    #############待加
    all_df_fea=pd.merge(all_df_fea,id_freq,on=['word'],how='left')
    ## 获取单词的逆词频
    
    ## 获取训练集和测试集
    train_df = all_df_fea[:len(train_df)]
    test_df = all_df_fea[len(train_df):]

    return train_df, test_df



tfidf_model, w2v_model, d2v_model = load_models(train_df)

def train_fea_make(train_index_all_list):
    fea_train=[]
    for train_index in tqdm(train_index_all_list):
        
        try:
            train_single = extract_word_property(train_df.iloc[train_index, :], tfidf_model, w2v_model, d2v_model, train=True)
            fea_train.append(train_single)
        except:
            continue
    return pd.concat(fea_train,axis=0)

def test_fea_make(test_index_all_list):
    fea_test=[]
    for test_index in test_index_all_list:
        try:
            test_single = extract_word_property(test_df.iloc[test_index, :], tfidf_model, w2v_model, d2v_model, train=False)

            fea_test.append(test_single)
        except:
            continue
    return pd.concat(fea_test,axis=0)

##############
pool = multiprocessing.Pool(processes=40)
train_fea=[]

split_size=int(len(train_df)/40)
all_index=list(range(len(train_df)))
df_split_index=[all_index[i:i+split_size] for i in range(0,len(all_index),split_size)]  

for part_id,part_index in enumerate(df_split_index):    
    train_fea.append(pool.apply_async(train_fea_make, [part_index]))
    print(part_id)
pool.close()
pool.join()
fea_train=[i.get() for i in train_fea]

print("模型训练完成")
#############


##############
pool = multiprocessing.Pool(processes=40)
test_fea=[]

split_size=int(len(test_df)/40)
all_index=list(range(len(test_df)))
df_split_index=[all_index[i:i+split_size] for i in range(0,len(all_index),split_size)]  

for part_id,part_index in enumerate(df_split_index):    
    test_fea.append(pool.apply_async(test_fea_make, [part_index]))
    print(part_id)
pool.close()
pool.join()
fea_test=[i.get() for i in test_fea]

print("模型训练完成")
#############




train_yuan=train_df
test_yuan=test_df

train_all2=pd.concat(fea_train).reset_index(drop=True)
test_all2=pd.concat(fea_test).reset_index(drop=True)

train_all, test_all = add_features(train_all2, test_all2)


# train_all=train_test[~train_test['label'].isnull()]
# test_all=train_test[train_test['label'].isnull()]


In [ ]:
test_all

In [ ]:
fea_train[0].columns

In [ ]:
train_all.to_csv("temp2/train_data.csv",index=None)
test_all.to_csv("temp2/test_data.csv",index=None)

# 加载已经做好的特征

In [4]:
import os
import math
import re
import datetime
import sys, getopt


import jieba
import re
import pandas as pd
import re
import os
import sys
import numpy as np
import string
from random import choice
import jieba
train_all=pd.read_csv("temp2/train_data.csv")
test_all=pd.read_csv("temp2/test_data.csv")

# 在训练前对特征进行预处理

## 去掉那些干扰词：在手动改训练集的时候，引入了一些 京东淘宝之类的，进去

In [21]:
train_all.loc[train_all['word']=='京东','label']=0
train_all.loc[train_all['word']=='阿里巴巴','label']=0
train_all.loc[train_all['word']=='莱特币','label']=0
train_all.loc[train_all['word']=='蚂蚁金服','label']=0
train_all.loc[train_all['word']=='唯品会','label']=0
train_all.loc[train_all['word']=='淘宝','label']=0
train_all.loc[train_all['word']=='天猫','label']=0

# 特征更新

In [6]:
all_df=pd.read_csv("temp2/all_tokens.csv")

# ctr_fea

In [18]:
# train_all=train_all.drop(['occr_times','select_times','ent_ctr'],axis=1)
# test_all=test_all.drop(['occr_times','select_times','ent_ctr'],axis=1)
ctr_df=pd.read_csv("data/ctr_over_four.csv")
ctr_df.columns=['word','occr_times','select_times','ent_ctr']
train_all=pd.merge(train_all,ctr_df,on=['word'],how='left')
train_all
test_all=pd.merge(test_all,ctr_df,on=['word'],how='left')
test_all
train_all

,word,cixing,neighbour,hits_h,hits_a,degrees,components,pagerank,doc_id,lda_classes,...,tf_index,word_distance_norm_index,cosine_index,euclidean_index,label,idf,idf_in_all_text,occr_times,select_times,ent_ctr
0,钱嗨娱乐,28,233.0,3.202750,3.202750,233.0,101329.0,6.660880,0,4,...,26,17,6,6,1.0,16,8.966938,16.0,16.0,0.959648
1,躺赚,28,11535.0,83.274707,83.274707,11535.0,101329.0,120.246907,0,4,...,13,3,14,16,1.0,278,5.045692,141.0,2.0,0.014418
2,明星周末,28,27.0,0.243519,0.243519,27.0,101329.0,2.470474,0,4,...,0,18,21,21,0.0,1,13.288866,NaN,NaN,NaN
3,全明星,15,130.0,2.238660,2.238660,130.0,101329.0,3.493467,0,4,...,1,13,15,4,0.0,2,12.703904,NaN,NaN,NaN
4,钱嗨,28,239.0,3.296492,3.296492,239.0,101329.0,6.713562,0,4,...,25,8,9,3,0.0,17,8.931314,16.0,0.0,0.002593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430060,交易,13,69297.0,195.982491,195.982491,69297.0,101329.0,915.531194,5005,5,...,4,4,8,8,0.0,4276,1.360718,1815.0,0.0,0.000024
430061,中国PE基金,28,8.0,0.154935,0.154935,8.0,101329.0,1.739767,5005,5,...,5,5,1,0,0.0,1,13.288866,NaN,NaN,NaN
430062,PE基金,28,229.0,3.434037,3.434037,229.0,101329.0,3.972357,5005,5,...,7,7,5,2,0.0,3,12.288866,NaN,NaN,NaN
430063,中国P,28,901.0,12.265358,12.265358,901.0,101329.0,10.534478,5005,5,...,8,8,2,3,0.0,7,10.381975,6.0,0.0,0.006453


In [ ]:
# train_all[['word','doc_id','pagerank','hits_h','neighbour','ent_ctr','occr_times','select_times']].to_csv("train_need_group_fea.csv",index=None)
# test_all[['word','doc_id','pagerank','hits_h','neighbour','ent_ctr','occr_times','select_times']].to_csv("test_need_group_fea.csv",index=None)

In [ ]:
train_all=train_all.drop(['over_05_times'],axis=1)
test_all=test_all.drop(['over_05_times'],axis=1)
# test_over_05=pd.read_csv("data/over_05.csv")
# train_all=pd.merge(train_all,test_over_05,on=['word'],how='left')
# train_all
# test_all=pd.merge(test_all,test_over_05,on=['word'],how='left')
# test_all

In [ ]:
drop_fea=['ctr_group_rank','select_times_group_rank','select_times_group_avg'
         ,'ctr_group_avg','select_times_group_avg']
train_all=train_all.drop(drop_fea,axis=1)
test_all=test_all.drop(drop_fea,axis=1)
# train_add_fea=pd.read_csv("add_fea/group_fea_add_train.csv")
# test_add_fea=pd.read_csv("add_fea/group_fea_add_test.csv")
# train_all=pd.merge(train_all,train_add_fea,on=['word','doc_id'],how='left')
# train_all
# test_all=pd.merge(test_all,test_add_fea,on=['word','doc_id'],how='left')
# test_all

In [ ]:
train_all.columns

# train

In [ ]:
# val_using_df=fea_train_df[fea_train_df['label']==1][0:20]
# val_using_df
# val_using_df=pd.concat([val_using_df,fea_train_df[fea_train_df['label']==0][0:20]])
# train_all['label']=train_all['label'].astype(int)
# train_all=train_all.reset_index(drop=True)
train_all

In [ ]:
fea_train_df_over_sam.sample(frac=1)

In [125]:
fea_train_df_over_sam[fea_train_df_over_sam['label']==1]

,word,cixing,neighbour,hits_h,hits_a,degrees,components,pagerank,doc_id,lda_classes,...,tf_index,word_distance_norm_index,cosine_index,euclidean_index,label,idf,idf_in_all_text,occr_times,select_times,ent_ctr
0,钱嗨娱乐,28,233.0,3.202750,3.202750,233.0,101329.0,6.660880,0,4,...,26,17,6,6,1.0,16,8.966938,16.0,16.0,0.938298
1,躺赚,28,11535.0,83.274707,83.274707,11535.0,101329.0,120.246907,0,4,...,13,3,14,16,1.0,278,5.045692,141.0,2.0,0.014413
28,钱嗨娱乐,28,233.0,3.202750,3.202750,233.0,101329.0,6.660880,1,4,...,1,11,1,3,1.0,16,8.966938,16.0,16.0,0.938298
29,躺赚,28,11535.0,83.274707,83.274707,11535.0,101329.0,120.246907,1,4,...,13,4,9,11,1.0,278,5.045692,141.0,2.0,0.014413
44,上证50etf期权,28,33.0,0.539496,0.539496,33.0,101329.0,1.849637,2,3,...,3,8,1,1,1.0,1,13.288866,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430015,启航,26,7510.0,71.275247,71.275247,7510.0,101329.0,64.845111,5002,7,...,8,8,6,3,1.0,58,7.547399,31.0,2.0,0.063798
430025,ing,28,25158.0,138.128603,138.128603,25158.0,101329.0,230.100278,5003,5,...,12,3,9,15,1.0,423,4.858414,181.0,1.0,0.005756
430026,金融服务,13,22769.0,116.528001,116.528001,22769.0,101329.0,238.894914,5003,5,...,20,20,6,14,1.0,566,4.112693,308.0,23.0,0.074564
430046,rnsfx,28,9.0,0.179206,0.179206,9.0,101329.0,1.580474,5004,5,...,1,1,2,1,1.0,1,13.288866,NaN,NaN,NaN


In [19]:
import lightgbm as lgb
from sklearn.metrics import f1_score
fea_train_df=train_all
fea=list(fea_train_df.columns)
fea.remove("label")
fea.remove("doc_id")
fea.remove("word")
# fea.remove("occr_times")[fea_train_df['label']==1]
# fea.remove("select_times")
# fea.remove("ent_ctr")
# 'neighbour', 'hits_h', 'hits_a', 'degrees', 'components', 'pagerank',
#fea=[i for i in fea if i not in ['neighbour', 'hits_h', 'hits_a', 'degrees', 
#                                 'components', 'pagerank']]
val_using_df=fea_train_df[fea_train_df['label']==1][0:20]
# val_using_df=fea_train_df[fea_train_df['label']==1]
val_using_df=pd.concat([val_using_df,fea_train_df[0:20]])
fea_train_df_over_sam=pd.concat([fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                 fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                 fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],

                                fea_train_df],axis=0)
# fea_train_df_over_sam=fea_train_df

# fea_train_df_over_sam=fea_train_df_over_sam.sample(frac=1)
train_x=fea_train_df_over_sam[fea]
train_y=fea_train_df_over_sam['label']



X_train = train_x
y_train = train_y
X_val = val_using_df[fea]
y_val = val_using_df['label']


params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'}, 
    'num_leaves': 45,
    'max_depth': -1,
     'subsample_freq':1,
     'feature_fraction':0.8,
    'learning_rate': 0.01,
    'colsample_bytree':0.8,
    'subsample':0.9,
     'reg_alpha':0.0,
     'reg_lambda':1,
    'min_child_weight':40,
    
}



print("start lgb")
# cate_fra=['cixing','stop_in_word','lda_classes','kmeans_classes']
lgb_train = lgb.Dataset(train_x,train_y)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=2500,
                valid_sets=lgb_eval,
                early_stopping_rounds=4000,
               verbose_eval=30)


preds = gbm.predict(X_val, num_iteration=gbm.best_iteration)
pre=[]
for i in preds:
    if i>0.4:
        pre.append(1)
    else:
        pre.append(0)
f1_score(y_val,pre)

start lgb
Training until validation scores don't improve for 4000 rounds.
[30]	valid_0's auc: 1
[60]	valid_0's auc: 1
[90]	valid_0's auc: 1
[120]	valid_0's auc: 1
[150]	valid_0's auc: 1
[180]	valid_0's auc: 1
[210]	valid_0's auc: 1
[240]	valid_0's auc: 1
[270]	valid_0's auc: 1
[300]	valid_0's auc: 1
[330]	valid_0's auc: 1
[360]	valid_0's auc: 1
[390]	valid_0's auc: 1
[420]	valid_0's auc: 1
[450]	valid_0's auc: 1
[480]	valid_0's auc: 1
[510]	valid_0's auc: 1
[540]	valid_0's auc: 1
[570]	valid_0's auc: 1
[600]	valid_0's auc: 1
[630]	valid_0's auc: 1
[660]	valid_0's auc: 1
[690]	valid_0's auc: 1
[720]	valid_0's auc: 1
[750]	valid_0's auc: 1
[780]	valid_0's auc: 1
[810]	valid_0's auc: 1
[840]	valid_0's auc: 1
[870]	valid_0's auc: 1
[900]	valid_0's auc: 1
[930]	valid_0's auc: 1
[960]	valid_0's auc: 1
[990]	valid_0's auc: 1
[1020]	valid_0's auc: 1
[1050]	valid_0's auc: 1
[1080]	valid_0's auc: 1
[1110]	valid_0's auc: 1
[1140]	valid_0's auc: 1
[1170]	valid_0's auc: 1
[1200]	valid_0's auc: 1
[1

0.7096774193548387

In [20]:
importance = gbm.feature_importance()
names = fea
for index, im in enumerate(importance):
    string = names[index] + ', ' + str(im)
    print(string)

cixing, 2
neighbour, 2
hits_h, 7
hits_a, 1
degrees, 2
components, 0
pagerank, 6
lda_classes, 3
kmeans_classes, 4
lsi_0, 2
lsi_1, 0
lsi_2, 9
lsi_3, 2
lsi_4, 0
lsi_5, 0
lsi_6, 2
lsi_7, 1
lsi_8, 0
lsi_9, 1
lsi_10, 4
lsi_11, 0
lsi_12, 0
lsi_13, 2
lsi_14, 5
tf, 1
stop_in_word, 8
is_in_first_sent, 2
is_in_last_sent, 0
is_in_other_sent, 0
sen_num, 0
word_count_xiangdui_max, 4
word_count_xiangdui_min, 2
word_count_group_max, 0
word_count_group_min, 0
word_count_group_avg, 4
word_count_group_min_xiangdui_max, 0
all_word_count_group, 1
gongsi_count_group, 0
gongsi_self_div_max_index_group, 0
gongsi_self_div_min_index_group, 0
is_gongsi_in_title, 0
is_gongsi_in_first_two_sen, 0
gongsi_avg_index, 2
gongsi_max_count, 0
word_len, 0
word_len_ratio, 0
tf_ratio, 1
tfidf, 0
dai_in_word, 0
bao_in_word, 0
bi_in_word, 0
gongsi_in_word, 1
zhengquan_in_word, 0
kuohao_in_word, 0
text_rank, 1
text_rank_geshu, 0
text_rank_rank, 3
baohan_max_len, 0
baohan_min_len, 0
baohan_self_div_other_count, 0
baohan_self_div

## xgboost

In [116]:
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
# 训练全部数据
fea_train_df=train_all.sample(frac=1)
fea=list(fea_train_df.columns)
fea.remove("label")
fea.remove("doc_id")
fea.remove("word")
# fea.remove("occr_times")
# fea.remove("select_times")
# fea.remove("ent_ctr")
# 'neighbour', 'hits_h', 'hits_a', 'degrees', 'components', 'pagerank',
#fea=[i for i in fea if i not in ['neighbour', 'hits_h', 'hits_a', 'degrees', 
#                                 'components', 'pagerank']]

val_using_df=fea_train_df[fea_train_df['label']==1][0:20]
# val_using_df=fea_train_df[fea_train_df['label']==1]
val_using_df=pd.concat([val_using_df,fea_train_df[fea_train_df['label']==0][0:20]])
fea_train_df_over_sam=pd.concat([fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                 fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
          
                                fea_train_df],axis=0)
fea_train_df_over_sam=fea_train_df_over_sam.sample(frac=1)
train_x=fea_train_df_over_sam[fea]
train_y=fea_train_df_over_sam['label']

val_size=int(-len(train_x)*0.001)

X_train = train_x[:val_size].values
y_train = train_y[:val_size].values
X_val = val_using_df[fea].values
y_val = val_using_df['label'].values

params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',

    'eval_metric': 'logloss',

    'learning_rate': 0.0894,
    'max_depth': 9,
    'max_leaves': 20,

    'lambda': 2,
    'alpha': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'silent': 1,
    'seed': 2019,

    # 使用gpu
    'gpu_id': 0,
    'tree_method': 'gpu_hist'
}


X_test=test_all[fea].values
xgb_test=xgb.DMatrix(X_test)
pred = np.zeros(len(test_all[fea]))


## 进行K折训练

train_X, train_y = X_train, y_train
eval_X, eval_y = X_val, y_val

xgb_train = xgb.DMatrix(train_X, train_y)
xgb_eval = xgb.DMatrix(eval_X, eval_y)


model = xgb.train(
    params,
    xgb_train,
    num_boost_round=2000,
    evals=[(xgb_train, 'train'), (xgb_eval, 'eval')],
    early_stopping_rounds=1000,
    verbose_eval=100
)

# pred += model.predict(xgb_test)


[0]	train-logloss:0.62714	eval-logloss:0.643876
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 1000 rounds.
[100]	train-logloss:0.119728	eval-logloss:0.258903


KeyboardInterrupt: 

In [ ]:
importance = model.get_fscore().items()
importance=[i[1] for i in importance]
names = fea
for index, im in enumerate(importance):
    string = names[index] + ', ' + str(im)
    print(string)

## catboost

In [ ]:
!source activate tf_bisai

In [ ]:
import catboost as ctb


from sklearn.metrics import f1_score
fea_train_df=train_all.sample(frac=1)
fea=list(fea_train_df.columns)
fea.remove("label")
fea.remove("doc_id")
fea.remove("word")
# fea.remove("occr_times")
# fea.remove("select_times")
# fea.remove("ent_ctr")
# 'neighbour', 'hits_h', 'hits_a', 'degrees', 'components', 'pagerank',
#fea=[i for i in fea if i not in ['neighbour', 'hits_h', 'hits_a', 'degrees', 
#                                 'components', 'pagerank']]
val_using_df=fea_train_df[fea_train_df['label']==1][0:20]
# val_using_df=fea_train_df[fea_train_df['label']==1]
val_using_df=pd.concat([val_using_df,fea_train_df[fea_train_df['label']==0][0:20]])
fea_train_df_over_sam=pd.concat([fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                 fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                 fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                 fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                 fea_train_df[fea_train_df['label']==1],fea_train_df[fea_train_df['label']==1],
                                fea_train_df[fea_train_df['label']==0]],axis=0)



fea_train_df_over_sam=fea_train_df_over_sam.sample(frac=1)
train_x=fea_train_df_over_sam[fea]
train_y=fea_train_df_over_sam['label']

val_size=int(-len(train_x)*0.001)

X_train = train_x[:val_size]
y_train = train_y[:val_size]
X_val = val_using_df[fea]
y_val = val_using_df['label']


ctb_model = ctb.CatBoostClassifier(iterations=500,learning_rate=0.05,max_depth=11,l2_leaf_reg=1,verbose=10)
ctb_model.fit(train_x,train_y,eval_set=(X_val,y_val))







In [ ]:
importance = ctb_model.feature_importances_

test_pre = ctb_model.predict_proba(test_features)[:,1]


importance = gbm.feature_importance()
names = fea
for index, im in enumerate(importance):
    string = names[index] + ', ' + str(im)
    print(string)

# 预测

## lgb

In [137]:

def get_is_new(ent_msg):
    preds = gbm.predict(ent_msg, num_iteration=gbm.best_iteration)
    pre=[]
    for i in preds:
        if i>0.4:
            pre.append(1)
        else:
            pre.append(0)
    return pre

result=get_is_new(test_all[fea])

# def get_is_new(ent_msg):
#     preds = gbm.predict(ent_msg, num_iteration=gbm.best_iteration)
#     pre=[i for i in preds]
#     pre=pd.DataFrame(pre)
#     return pre

# result=get_is_new(test_all[fea])
# result=pd.concat([test_all['word'],result],axis=1)
# result.to_csv("test_score.csv",index=None)

## xgb

In [ ]:
# def get_is_new(preds):
#     pre=[]
#     for i in preds:
#         if i>0.5:
#             pre.append(1)
#         else:
#             pre.append(0)
#     return pre
# result=get_is_new(pred)

## ronghe

In [ ]:
preds_lgb = gbm.predict(test_all[fea], num_iteration=gbm.best_iteration)
preds_lgb=[i for i in preds_lgb]
preds_xgb = model.predict(xgb_test)
preds_xgb=[i for i in preds_xgb]

In [ ]:
result=[(i+j)/2 for i,j in zip(preds_lgb,preds_xgb)]
result=[1 if i>0.3 else 0 for i in result]

In [138]:
old_entities = []
train_df = pd.read_csv("data/Round2_train.csv", encoding="utf-8-sig")
for x in list(train_df["unknownEntities"].fillna("")):
    old_entities.extend(x.split(";"))
old_entities = set(old_entities)
add_char = {']', '：', '~', '！', '%', '[', '《', '】', ';', '”', ':', '》', '？', '>', '/', '#', '。', '；', '&', '=', '，', '“', '【'}

def islegitimate(x):
    if re.findall("\\"+"|\\".join(add_char), x):
        return False
    if x in old_entities:
        return False

    return True

result_all=pd.DataFrame()
result_all['result']=result
result_all['word']=test_all['word'].reset_index(drop=True)
result_all['doc_id']=test_all['doc_id'].reset_index(drop=True)
result_all=result_all[result_all['result']==1]
result_all=result_all[['doc_id','word']]
result_all=result_all.values.tolist()
result_df=[[] for i in range(test_r2_len)]
for i in result_all:
    if islegitimate(i[1]):
        result_df[i[0]-10019].append(i[1])
result_df=[';'.join(list(set(i))) if i !=[] else np.nan for i in result_df]
result_df=pd.DataFrame(result_df)
# 


id_df=pd.read_csv('data/Round2_Test.csv',usecols=['id'])
result_df=pd.concat([id_df,result_df],axis=1)
result_df.columns=['id','unknownEntities']
result_df

,id,unknownEntities
0,1,芒果小V;芒果达人;淘宝
1,2,四川恩威集团;恩威道源商城;恩威集团;恩威道源
2,3,月月救急
3,4,墨菲;素店晨晨;晨晨
4,5,民间投资;insisted;拍雅安;民间投;钱旺实业;Qbo网
...,...,...
4994,5000,前海云轩互联网金融服务有限公司;原沪深理财;深圳前海云轩互联网金融服务有限公司;理财官网;首...
4995,5001,阿里巴巴;BHEX;顶级投资;TokenFund;BHEX平台;BHT;了得资本;Blueh...
4996,5002,Agen;期货投资平台;移动交易;文华财经;逸富国际期货交易;华尔街交易员;国际期货交易;金...
4997,5003,想排油;广药王老吉;anta522;广药吉悠


In [139]:
result_df.to_csv("new_submit/all_bert.csv",index=None)

In [ ]:
result_all